Необхідні імпорти

In [1]:
import re
import chardet
import math
from collections import Counter
import itertools
import pandas as pd

Глобальні константи

In [ ]:
ALPHABET = "абвгдежзийклмнопрстуфхцчшщьыэюя"
m = len(ALPHABET)
N = m * m  

CHAR_TO_NUM = {char: i for i, char in enumerate(ALPHABET)}
NUM_TO_CHAR = {i: char for i, char in enumerate(ALPHABET)}

PLAINTEXT_TOP5 = ["ст", "но", "то", "на", "ен"]

print(f"Алфавіт: {ALPHABET}")
print(f"Потужність алфавіту (m): {m}")
print(f"Простір біграм (N = m^2): {N}")

Алфавіт: абвгдежзийклмнопрстуфхцчшщьыэюя
Потужність алфавіту (m): 31
Простір біграм (N = m^2): 961


Обробка тексту з першої лаби

In [3]:
def obrobka_text(filename, remove_spaces=True):
    try:
        with open(filename, "rb") as f:
            raw_data = f.read()
        koduvannya = chardet.detect(raw_data)["encoding"]
        if koduvannya is None:
            koduvannya = 'utf-8' 
            
        with open(filename, "r", encoding=koduvannya, errors="replace") as file:
            text = file.read().lower()
        
        text = text.replace("ё", "е").replace("ъ", "ь")
        text = re.sub(r'[^а-я ]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        if remove_spaces:
            text = text.replace(" ", "")
            
        return text
        
    except FileNotFoundError:
        print(f"ПОМИЛКА: Файл {filename} не знайдено.")
        return None
    except Exception as e:
        print(f"ПОМИЛКА при читанні файлу: {e}")
        return None

Арифметика 

In [4]:
#Розширений алгоритм Евкліда: повертає (gcd, x, y)
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

#Знаходить обернений елемент до 'a' за модулем 'm'
def mod_inverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        return None  
    else:
        return x % m

#Розв'язує порівняння ax ≡ b (mod n)
def solve_linear_congruence(a, b, n):
    a = a % n
    b = b % n
    g, x_g, y_g = egcd(a, n)
    
    if b % g != 0:
        return []  
    
    #Знаходимо один розв'язок
    a1 = a // g
    b1 = b // g
    n1 = n // g
    x0 = (b1 * mod_inverse(a1, n1)) % n1
    
    #Знаходимо всі d розв'язків 
    solutions = []
    for i in range(g):
        solutions.append((x0 + i * n1) % n)
        
    return solutions

Перетворення біграм

In [5]:
#Перетворює біграму (напр., 'ст') в число
def bigram_to_num(bg):
    if len(bg) != 2 or bg[0] not in CHAR_TO_NUM or bg[1] not in CHAR_TO_NUM:
        return None
    x1 = CHAR_TO_NUM[bg[0]]
    x2 = CHAR_TO_NUM[bg[1]]
    return x1 * m + x2

#Перетворює число (0..N-1) в біграму
def num_to_bigram(num):
    x1 = num // m
    x2 = num % m
    return NUM_TO_CHAR[x1] + NUM_TO_CHAR[x2]

Основні функції криптоаналізу

In [6]:
def chastoty_bigram_non_overlapping(tekst):
    bigrams = [tekst[i:i+2] for i in range(0, len(tekst) - 1, 2)]
    N_bg = len(bigrams)
    counts = Counter(bigrams)
    freq = {bg: count / N_bg for bg, count in counts.items()}
    return freq, counts, N_bg

def decrypt(ciphertext, a, b):
    a_inv = mod_inverse(a, N)
    if a_inv is None:
        return ""  
        
    plaintext = ""
    bigrams = [ciphertext[i:i+2] for i in range(0, len(ciphertext) - 1, 2)]
    
    for bg in bigrams:
        Y = bigram_to_num(bg)
        if Y is None:
            continue
            
        X = (a_inv * (Y - b + N)) % N 
        plaintext += num_to_bigram(X)
        
    return plaintext

def is_meaningful(text):
    if not text:
        return False
        
    counts = Counter(text)
    N_text = len(text)
    freq = {char: count / N_text for char, count in counts.items()}
    
    # Перевірка частот найчастіших літер 
    common_freq_sum = freq.get('о', 0) + freq.get('а', 0) + freq.get('е', 0)
    
    if common_freq_sum >= 0.25: 
        return True
        
    return False

Головна функція

In [ ]:
def main_attack(ciphertext_file):
    
    BIGRAM_EXCEL_FILE = "ciphertext_non_overlapping_bigrams.xlsx"
    ALL_KEYS_FILE = "all_potential_keys.txt"
    ALL_DECRYPTIONS_FILE = "all_decryptions.txt"
    CORRECT_DECRYPT_FILE = "decrypted_correct_key.txt"
    
    print(f"--- Початок криптоаналізу файлу {ciphertext_file} ---")
    
    print("\n[Крок 1] Обробка тексту та підрахунок частот...")
    ciphertext = obrobka_text(ciphertext_file, remove_spaces=True)
    if ciphertext is None:
        print("Аналіз неможливий. Файл не завантажено.")
        return
        
    print(f"  > Текст успішно завантажено, довжина: {len(ciphertext)} символів.")
    

    print(f"  > (INFO: Використовуються біграми, що не перетинаються )")
    freq_cipher, counts_cipher, n_cipher = chastoty_bigram_non_overlapping(ciphertext)
    
    top_n = 5
    top_cipher_bg = [
        bg for bg, _ in sorted(
            counts_cipher.items(), key=lambda item: item[1], reverse=True
        ) if len(bg) == 2
    ][:top_n]
    
    if len(top_cipher_bg) < 2:
        print("  > ПОМИЛКА: В шифртексті знайдено менше 2 біграм. Аналіз неможливий.")
        return
    
    print(f"\n[Крок 2] Знайдені {top_n} найчастіших біграм шифртексту:")
    print(f"  > {top_cipher_bg}")
    print(f"  > Відомі {top_n} найчастіших біграм відкритого тексту:")
    print(f"  > {PLAINTEXT_TOP5}")

    try:
        df_bigrams = pd.DataFrame(
            [(bg, counts_cipher[bg], freq_cipher[bg]) for bg in counts_cipher if len(bg) == 2],
            columns=["Біграма", "Кількість", "Частота"]
        ).sort_values(by="Частота", ascending=False)
        
        df_bigrams.to_excel(BIGRAM_EXCEL_FILE, index=False)
        print(f"\n  > Збережено Excel-файл з усіма біграмами: {BIGRAM_EXCEL_FILE}")
    except Exception as e:
        print(f"  > ПОМИЛКА при збереженні Excel-файлу: {e}")

    X_top = [bigram_to_num(bg) for bg in PLAINTEXT_TOP5]
    Y_top = [bigram_to_num(bg) for bg in top_cipher_bg]
    
    print("\n[Кроки 3-5] Пошук ключа та дешифрування...")
    print(f"  > Потенційні ключі будуть записані у {ALL_KEYS_FILE}")
    print(f"  > Результати дешифрування будуть у {ALL_DECRYPTIONS_FILE}")
    
    with open(ALL_KEYS_FILE, "w", encoding="utf-8") as f_keys, \
         open(ALL_DECRYPTIONS_FILE, "w", encoding="utf-8") as f_decryptions:
        
        found_key = False
        tested_keys = set() 

        for (i, j) in itertools.permutations(range(top_n), 2):
            X_star = X_top[i]
            X_star_star = X_top[j]
            

            for (k, l) in itertools.permutations(range(top_n), 2):
                Y_star = Y_top[k]
                Y_star_star = Y_top[l]

                delta_X = (X_star - X_star_star + N) % N
                delta_Y = (Y_star - Y_star_star + N) % N

                possible_as = solve_linear_congruence(delta_X, delta_Y, N)
                
                if not possible_as:
                    continue
                    
                for a in possible_as:
                    if math.gcd(a, m) != 1: 
                        continue
                        
                    b = (Y_star - (a * X_star) + N) % N
                    
                    key = (a, b)
                    if key in tested_keys:
                        continue 
                    tested_keys.add(key)
                    
                    f_keys.write(f"{a},{b}\n")

                    plaintext = decrypt(ciphertext, a, b)
                    
                    f_decryptions.write(f"--- Key (a, b): ({a}, {b}) ---\n")
                    f_decryptions.write(f"Припущення: '{PLAINTEXT_TOP5[i]}' -> '{top_cipher_bg[k]}' та '{PLAINTEXT_TOP5[j]}' -> '{top_cipher_bg[l]}'\n")
                    f_decryptions.write(f"{plaintext[:200]}...\n\n")

                    if not found_key and is_meaningful(plaintext):
                        found_key = True 
                        
                        print("\n" + "="*70)
                        print("[Крок 5] ЗНАЙДЕНО ЙМОВІРНИЙ КЛЮЧ")
                        print("="*70)
                        print(f"  > Припущення: '{PLAINTEXT_TOP5[i]}' ('{X_star}') -> '{top_cipher_bg[k]}' ('{Y_star}')")
                        print(f"  >           та '{PLAINTEXT_TOP5[j]}' ('{X_star_star}') -> '{top_cipher_bg[l]}' ('{Y_star_star}')")
                        print(f"  > Знайдене значення ключа (a, b): ({a}, {b})")
                        print("\n  --- Розшифрований текст (перші 500 символів) ---")
                        print(f"  {plaintext[:500]}...")
                        
                        try:
                            with open(CORRECT_DECRYPT_FILE, "w", encoding="utf-8") as f_correct:
                                f_correct.write(f"Знайдено з ключем (a, b): ({a}, {b})\n")
                                f_correct.write(f"Припущення: '{PLAINTEXT_TOP5[i]}' -> '{top_cipher_bg[k]}' та '{PLAINTEXT_TOP5[j]}' -> '{top_cipher_bg[l]}'\n")
                                f_correct.write("="*70 + "\n")
                                f_correct.write(plaintext)
                            print(f"\n  !!! Повний розшифрований текст збережено у {CORRECT_DECRYPT_FILE} !!!")
                        except Exception as e:
                            print(f"  > ПОМИЛКА при збереженні правильного тексту: {e}")
                        print("="*70)

    if not found_key:
        print("\n[Результат] Перебір завершено, але змістовний текст не знайдено.")
        print("  > Перевірте ваш файл з варіантом або критерії в 'is_meaningful'.")
    else:
        print(f"\n[Результат] Перебір завершено. Знайдено щонайменше один ймовірний ключ.")
    print("--- Аналіз завершено ---")

Запуск аналізу

In [8]:
CIPHERTEXT_FILE_PATH = "03.txt" 

main_attack(CIPHERTEXT_FILE_PATH)

--- Початок криптоаналізу файлу 03.txt ---

[Крок 1] Обробка тексту та підрахунок частот...
  > Текст успішно завантажено, довжина: 5630 символів.
  > (INFO: Використовуються біграми, що не перетинаються )

[Крок 2] Знайдені 5 найчастіших біграм шифртексту:
  > ['тд', 'рб', 'во', 'щю', 'кд']
  > Відомі 5 найчастіших біграм відкритого тексту:
  > ['ст', 'но', 'то', 'на', 'ен']

  > Збережено Excel-файл з усіма біграмами: ciphertext_non_overlapping_bigrams.xlsx

[Кроки 3-5] Пошук ключа та дешифрування...
  > Потенційні ключі будуть записані у all_potential_keys.txt
  > Результати дешифрування будуть у all_decryptions.txt

[Крок 5] ЗНАЙДЕНО ЙМОВІРНИЙ КЛЮЧ
  > Припущення: 'ст' ('545') -> 'тд' ('562')
  >           та 'но' ('417') -> 'во' ('76')
  > Знайдене значення ключа (a, b): (199, 700)

  --- Розшифрований текст (перші 500 символів) ---
  отцеубийствокакизвестноосновноеиизначальноепреступлениечеловечестваиотдельногочеловекавовсякомслучаеоноглавныйисточникчувствавинынеизвестноединствен